In [14]:
import numpy as np
import pandas as pd

df = pd.read_csv('marketing_campaign_data_messy.csv')

df.shape

(2020, 12)

In [15]:
df.head(2)

,Campaign_ID,Campaign_Name,Start_Date,End_Date,Channel,Impressions,Clicks,Spend,Conversions,Active,Clicks,Campaign_Tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,0,NaN,FA


In [16]:
# Step 1 : Header Cleaning

print(df.columns.tolist())

df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

print("Headers cleaned")
print(df.columns.tolist())

[' Campaign_ID ', 'Campaign_Name', 'Start_Date', 'End_Date', 'Channel', 'Impressions', 'Clicks ', 'Spend', 'Conversions', 'Active', 'Clicks', 'Campaign_Tag']
Headers cleaned
['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


##### doubt: in last line how did dirty_spend_mask get the $ symbol removed values?? we assigned it to the df['spend'] coulmn n not dirty_spend_mask column?

##### The dirty_spend_mask itself did not change. It is still just a list (a Boolean Series) of True and False values that remembers which row positions originally had a dollar sign.Think of the mask like a "bookmark" or a "pointer" to specific rows:Creation: You created the mask to identify rows where the currency symbol existed (e.g., Row 0, Row 21).The Change: You then modified the actual data inside df['spend'] globally.The Result: When you run df.loc[dirty_spend_mask, 'spend'] at the end, you are telling Pandas: "Go to the row positions I bookmarked earlier, and show me what is currently in the 'spend' column at those spots."Because you cleaned the column in the step right before, the "bookmarked" rows now show the cleaned numbers. The mask pointed to the location, and the data at that location was updated.


In [17]:
#Step 2 : Type conversion n Currency cleaning

dirty_spend_mask = df['spend'].astype(str).str.contains(r'\$')
print(df.loc[dirty_spend_mask,('campaign_id','spend')].head(3))

df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]','', regex = True)
df['spend'] = pd.to_numeric(df['spend'])

print("$ symbol removed from 'spend' column")
print(df.loc[dirty_spend_mask,('campaign_id','spend')].head(3))


   campaign_id     spend
0    CMP-00001   $102.82
21   CMP-00022   $2428.4
22   CMP-00023  $4726.22
$ symbol removed from 'spend' column
   campaign_id    spend
0    CMP-00001   102.82
21   CMP-00022  2428.40
22   CMP-00023  4726.22


In [18]:
#Step 3: Categorical typos

print(df['channel'].unique())

cleanup_map = {
    'Facebok' : 'Facebook',
    'Tik_Tok' : 'TikTok',
    'Insta_gram' : 'Instagram',
    'Gogle' : 'Google Ads',
    'E-mail' : 'Email',
    'N/A' : np.nan
}

df['channel'] = df['channel'].replace(cleanup_map)

print("Spelling errors corrected")
print(df['channel'].unique())

['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' 'E-mail' nan 'Gogle'
 'Tik_Tok' 'Facebok' 'Insta_gram']
Spelling errors corrected
['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' nan]


In [19]:
#Step 4 : Handling mixed boolean

print(df['active'].unique())

bool_map = {
    'Y': True,
    '0' : False, 
    'No' : False, 
    'Yes' : True , 
    '1' : True, 
    1 : True,
    0: False
}

df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)

print("Fix applied")
print(df['active'].unique())

['Y' '0' 'No' 'True' 'Yes' '1' 'False']
Fix applied
[ True False]


C:\Users\Kimay\AppData\Local\Temp\ipykernel_28504\1599116193.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)


In [20]:
#Step 5: Date Parsing

print(df['start_date'].dtype)

df['start_date'] = pd.to_datetime(df['start_date'], errors = 'coerce')
df['end_date'] = pd.to_datetime(df['end_date'], errors = 'coerce')

print("Dates converted to date time from object")
print(df['start_date'].dtype)

object
Dates converted to date time from object
datetime64[ns]


In [21]:
# removing duplicate columns

df = df.loc[:, ~df.columns.duplicated()]

In [22]:
#Step 6 : Checking if logic in data r correct (eg impression > clicks always)

impossible_mask = df['clicks'] > df['impressions']
print(df.loc[impossible_mask,['campaign_id']].head(3))

#this means there is no case where clicks>impression

Empty DataFrame
Columns: [campaign_id]
Index: []


In [23]:
#Step 7 : Logical Integrity (checking whether end date is after start date or not)

time_travel_mask = df['end_date'] < df['start_date']
print(df.loc[time_travel_mask, ['campaign_id','start_date','end_date']].head(3))

df.loc[time_travel_mask, 'end_date'] = df.loc[time_travel_mask, 'start_date'] + pd.Timedelta(days=30)

print("Fix applied")


   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-05-01
54   CMP-00055 2023-09-01 2023-08-27
71   CMP-00072 2023-02-01 2023-01-27
Fix applied


In [24]:
print(df.loc[time_travel_mask, ['campaign_id', 'start_date', 'end_date']].head(3))

   campaign_id start_date   end_date
23   CMP-00024 2023-05-06 2023-06-05
54   CMP-00055 2023-09-01 2023-10-01
71   CMP-00072 2023-02-01 2023-03-03


In [26]:
#step 8 : handling outliers (winsorizing)

Q1 = df['spend'].quantile(0.25)
Q3 = df['spend'].quantile(0.75)

IQR = Q3 - Q1
upper_limit = Q3 + (1.5*IQR)
lower_limit = Q1 - (1.5*IQR)

outlier_mask = (df['spend'] > upper_limit) | (df['spend'] < lower_limit)
print(df.loc[outlier_mask, ['campaign_id', 'spend']].head(3))

    campaign_id    spend
57    CMP-00058  5719.84
70    CMP-00071  5951.42
119   CMP-00120  7471.52


In [30]:
#Step 9 : String parsing (feature extraction)

print(df['campaign_name'].head(3))

df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_')
print("Fix applied")
print(df[['campaign_name','season']].head(3))

0    Q4_Summer_CMP-00001
1    Q1_Launch_CMP-00002
2    Q3_Winter_CMP-00003
Name: campaign_name, dtype: object
Fix applied
         campaign_name  season
0  Q4_Summer_CMP-00001  Summer
1  Q1_Launch_CMP-00002  Launch
2  Q3_Winter_CMP-00003  Winter
